In [2]:
import numpy as np
import pandas as pd
import csv
import os
import tabulate



In [3]:
from sklearn.preprocessing import LabelEncoder
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, roc_curve

In [4]:
#definimos el path donde se encuentre el archivo
path = "C:\\Users\\mabett\\Downloads\\ltm\\"
#os.chdir(path)
ddbb = pd.read_csv(path+"dataset_SCL.csv")

C:\Users\mabett\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
ddbb[ddbb["AÑO"]==2018].describe()

,DIA,MES,AÑO
count,2.0,2.0,2.0
mean,1.0,1.0,2018.0
std,0.0,0.0,0.0
min,1.0,1.0,2018.0
25%,1.0,1.0,2018.0
50%,1.0,1.0,2018.0
75%,1.0,1.0,2018.0
max,1.0,1.0,2018.0


In [51]:
ddbb.tail()

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,same_1,same_2,same_3
68201,2017-12-22 14:55:00,400,SCEL,SPJC,JAT,2017-12-22 15:41:00,400.0,SCEL,SPJC,JAT,...,12,2017,Viernes,I,JetSmart SPA,Santiago,Lima,1,1,1
68202,2017-12-25 14:55:00,400,SCEL,SPJC,JAT,2017-12-25 15:11:00,400.0,SCEL,SPJC,JAT,...,12,2017,Lunes,I,JetSmart SPA,Santiago,Lima,1,1,1
68203,2017-12-27 14:55:00,400,SCEL,SPJC,JAT,2017-12-27 15:35:00,400.0,SCEL,SPJC,JAT,...,12,2017,Miercoles,I,JetSmart SPA,Santiago,Lima,1,1,1
68204,2017-12-29 14:55:00,400,SCEL,SPJC,JAT,2017-12-29 15:08:00,400.0,SCEL,SPJC,JAT,...,12,2017,Viernes,I,JetSmart SPA,Santiago,Lima,1,1,1
68205,2017-12-31 14:55:00,400,SCEL,SPJC,JAT,2017-12-31 15:04:00,400.0,SCEL,SPJC,JAT,...,12,2017,Domingo,I,JetSmart SPA,Santiago,Lima,1,1,1


In [15]:
#Podemos ver que hay variables que no aportan informacion en primera instancia
#Primer PASO: entender bien el problema y que es lo que buscamos
#Partimos con un EDA, donde vemos el tipo de variables, como se comportan, si hay na/null, ya que si la data es mala, garbage in garbage out
#Cosas interesantes: verificar si el destino programado y el vuelo/aerolinea son iguales al que paso finalmente
#Hay meses, dias de la semana y horas que tienen mayor demanda, por lo que afectan la probabilidad de delay del vuelo
#Hay destinos mas concurridos que tambien pueden afectar en el delay
ddbb.describe()
ddbb.info()
ddbb["Emp-I"].unique()
#ddbb.isna().sum()
ddbb.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68206 entries, 0 to 68205
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Fecha-I    68206 non-null  object
 1   Vlo-I      68206 non-null  object
 2   Ori-I      68206 non-null  object
 3   Des-I      68206 non-null  object
 4   Emp-I      68206 non-null  object
 5   Fecha-O    68206 non-null  object
 6   Vlo-O      68205 non-null  object
 7   Ori-O      68206 non-null  object
 8   Des-O      68206 non-null  object
 9   Emp-O      68206 non-null  object
 10  DIA        68206 non-null  int64 
 11  MES        68206 non-null  int64 
 12  AÑO        68206 non-null  int64 
 13  DIANOM     68206 non-null  object
 14  TIPOVUELO  68206 non-null  object
 15  OPERA      68206 non-null  object
 16  SIGLAORI   68206 non-null  object
 17  SIGLADES   68206 non-null  object
dtypes: int64(3), object(15)
memory usage: 9.4+ MB


Fecha-I      0
Vlo-I        0
Ori-I        0
Des-I        0
Emp-I        0
Fecha-O      0
Vlo-O        1
Ori-O        0
Des-O        0
Emp-O        0
DIA          0
MES          0
AÑO          0
DIANOM       0
TIPOVUELO    0
OPERA        0
SIGLAORI     0
SIGLADES     0
dtype: int64

In [25]:
ddbb["same_1"] = [1 if ddbb["Des-I"][i]==ddbb["Des-O"][i] else 0 for i in range(len(ddbb))]
ddbb["same_2"] = [1 if ddbb["Vlo-I"][i]==ddbb["Vlo-O"][i] else 0 for i in range(len(ddbb))]
ddbb["same_3"] = [1 if ddbb["Emp-I"][i]==ddbb["Emp-O"][i] else 0 for i in range(len(ddbb))]

In [26]:
#En un 27% de los casos la compañia del vuelo programado cambia con respecto al final
ddbb.same_1.unique()
a1 = ddbb.same_1.values.sum()
a2 = ddbb.same_2.values.sum()
a3 = ddbb.same_3.values.sum()
a4 = len(ddbb.same_1)
t1 = (a4-a1)/a4
t2 = (a4-a2)/a4
t3 = (a4-a3)/a4
print(t1,t2,t3)

0.0004105210685276955 0.027123713456294166 0.273714922440841


In [76]:
db1 = ddbb[ddbb.same_3==0].reset_index(drop=True)
db2 = db1.groupby("TIPOVUELO")["DIA"].count().reset_index()
db2
db3 = db1.groupby("Emp-O")["DIA"].count().reset_index()
db3
db4 = db1[db1["Emp-O"]=="LXP"].reset_index(drop=True)
db4["Emp-O"].unique()
# De este 27% la verdad es que es un cambio substancial ya que la empresa sigue siendo la misma (LAN = LXP)


array(['LXP'], dtype=object)

In [30]:
ddbb.groupby("TIPOVUELO")["DIA"].count().reset_index()
#La cantidad de vuelos nacionales vs internacionales es similar

,TIPOVUELO,DIA
0,I,31240
1,N,36966


In [32]:
ddbb.describe()
ddbb.columns.to_list()

['Fecha-I',
 'Vlo-I',
 'Ori-I',
 'Des-I',
 'Emp-I',
 'Fecha-O',
 'Vlo-O',
 'Ori-O',
 'Des-O',
 'Emp-O',
 'DIA',
 'MES',
 'AÑO',
 'DIANOM',
 'TIPOVUELO',
 'OPERA',
 'SIGLAORI',
 'SIGLADES',
 'same_1',
 'same_2',
 'same_3']

In [ ]:
#Veamos algunos graficos previo a transformar las variables
attributes = ['Fecha-I','Vlo-I','Ori-I','Des-I','Emp-I','Fecha-O','Vlo-O','Ori-O','Des-O', 'Emp-O', 'DIA','MES',
'AÑO','DIANOM', 'TIPOVUELO', 'OPERA', 'SIGLAORI', 'SIGLADES', 'same_1', 'same_2', 'same_3']
for i,pred in enumerate(attributes):
    plt.figure(i, figsize=(15, 5))
    group = data.groupby([pred], as_index=False).aggregate(np.mean)[[pred, 'ArrDel15']]
    group.sort_values(by=pred, inplace=True)    
    group.plot.bar(x=pred, y='ArrDel15')
    plt.axhline(y=avgLate, label='Average')
    plt.ylabel('Percent of Flights that Arrive Late')
    plt.title(pred)
    plt.legend().remove()

In [39]:
#Veamos algunos graficos previo a transformar las variables
group = ddbb.groupby(["MES","TIPOVUELO"], as_index=False)["DIA"].count()
group.head()
group = group.rename(columns={"DIA":"Vuelos"})
#group.sort_values(by=co, inplace=True)    
import plotly.express as px
fig = px.bar(group, x="MES", y="Vuelos", color="TIPOVUELO", barmode="group")
fig.show()

In [46]:
group = ddbb.groupby(["MES","TIPOVUELO","DIANOM","AÑO","OPERA"], as_index=False)["DIA"].count()
group.head()
group = group.rename(columns={"DIA":"Vuelos"})
  
#fig = px.bar(group, x="MES", y="Vuelos", color="TIPOVUELO", barmode="group")

fig = px.bar(group, x="DIANOM", y="Vuelos", color="TIPOVUELO", barmode="group")
fig.show()

In [49]:
fig = px.bar(group, x="OPERA", y="Vuelos", color="TIPOVUELO", barmode="group")
fig.show()
#La mayor concentración de vuelos los tiene latam y sky

In [56]:
fig = px.bar(group, x="DIANOM", y="Vuelos")
fig.show()

In [47]:
#A considerar: para entrenar bien el modelo las clases deben estar balanceadas (evitar under/overfitting)
classDistribution = fdata['ArrDel15'].value_counts()
print('Class imbalance:')
print(classDistribution)
zero = fdata[fdata['ArrDel15']==0].tail(classDistribution.min())
one = fdata[fdata['ArrDel15']==1]
data = zero.append(one)
del zero, one
data.sort(['Year','Month','DayofMonth','DayOfWeek'], ascending=[False,False,False,False])
#del fdata
print('Class imbalance evened out:')
print(data['ArrDel15'].value_counts())
len(data.columns)



NameError: name 'fdata' is not defined

In [18]:
### Creo las nuevas variables ###
#temporada_alta : 1 si Fecha-I está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep, 0 si no.
ddbb.columns
ddbb.head()
## Pasamos la fecha a formato datetime
ddbb["Fecha-I"] = pd.to_datetime(ddbb["Fecha-I"])
ddbb.head()
ddbb["temporada_alta"] = [1 if ddbb["Fecha-I"][i]<pd.Timestamp(2017,3,3) or ddbb["Fecha-I"][i]>pd.Timestamp(2017,12,15)or(ddbb["Fecha-I"][i]>pd.Timestamp(2017,7,15) and ddbb["Fecha-I"][i]<pd.Timestamp(2017,7,31))or
                         (ddbb["Fecha-I"][i]>pd.Timestamp(2017,9,11) and ddbb["Fecha-I"][i]<pd.Timestamp(2017,9,30)) else 0 for i in range(len(ddbb))]
#ddbb["Fecha-I"][0]

In [26]:
ddbb.head(70000)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,temporada_alta
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami,1
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami,1
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami,1
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami,1
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,2017-12-22 14:55:00,400,SCEL,SPJC,JAT,2017-12-22 15:41:00,400.0,SCEL,SPJC,JAT,22,12,2017,Viernes,I,JetSmart SPA,Santiago,Lima,1
68202,2017-12-25 14:55:00,400,SCEL,SPJC,JAT,2017-12-25 15:11:00,400.0,SCEL,SPJC,JAT,25,12,2017,Lunes,I,JetSmart SPA,Santiago,Lima,1
68203,2017-12-27 14:55:00,400,SCEL,SPJC,JAT,2017-12-27 15:35:00,400.0,SCEL,SPJC,JAT,27,12,2017,Miercoles,I,JetSmart SPA,Santiago,Lima,1
68204,2017-12-29 14:55:00,400,SCEL,SPJC,JAT,2017-12-29 15:08:00,400.0,SCEL,SPJC,JAT,29,12,2017,Viernes,I,JetSmart SPA,Santiago,Lima,1


In [63]:
### Calculo de retraso en minutos ###
ddbb.info()
ddbb["Fecha-O"] = pd.to_datetime(ddbb["Fecha-O"])
ddbb["dif"] = ddbb["Fecha-O"] - ddbb["Fecha-I"]
ddbb["dif_min"] = [ddbb.dif[i].total_seconds()/60 for i in range(len(ddbb))]
ddbb["atraso"] = [1 if ddbb.dif_min[i] > 15 else 0 for i in range(len(ddbb))]
ddbb.head(100)
ddbb["periodo_dia"] = ["mañana" if ddbb["Fecha-I"][i].hour in range(5,12) else "tarde" if ddbb["Fecha-I"][i].hour in range(12,19) else "noche" for i in range(len(ddbb))]


ddbb.groupby("periodo_dia")["atraso"].mean().reset_index()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68206 entries, 0 to 68205
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype          
---  ------          --------------  -----          
 0   Fecha-I         68206 non-null  datetime64[ns] 
 1   Vlo-I           68206 non-null  object         
 2   Ori-I           68206 non-null  object         
 3   Des-I           68206 non-null  object         
 4   Emp-I           68206 non-null  object         
 5   Fecha-O         68206 non-null  datetime64[ns] 
 6   Vlo-O           68205 non-null  object         
 7   Ori-O           68206 non-null  object         
 8   Des-O           68206 non-null  object         
 9   Emp-O           68206 non-null  object         
 10  DIA             68206 non-null  int64          
 11  MES             68206 non-null  int64          
 12  AÑO             68206 non-null  int64          
 13  DIANOM          68206 non-null  object         
 14  TIPOVUELO       68206 non-null  object

,periodo_dia,atraso
0,mañana,0.160066
1,noche,0.200023
2,tarde,0.199404


In [76]:
len(ddbb.SIGLADES.unique())
import plotly.express as px
ddbb.SIGLADES.unique()
grou = ddbb.groupby("SIGLADES")["atraso"].count().reset_index()
fig = px.bar(grou, x="SIGLADES", y="atraso")
fig.show()
print(ddbb.atraso.sum()/len(ddbb)*100)


## casi un 19% de los vuelos tienen atrasados en su hora de salida
## un dato relevante seria ver el promedio de los minutos atrasados, para clasificarlos por gravedad, ya que
## no es lo mismo 10 minutos versus 2 horas de atraso de cara al cliente.
## la distribucion de los atrasos tiene una caida exponencial, concetrda mas en los valores menores a 40 min
aux1 = ddbb[ddbb.atraso==1]
fig = px.histogram(aux1, "dif_min")
fig.show()
aux1.describe()


18.493974137172682


,DIA,MES,AÑO,temporada_alta,dif,dif_min,atraso
count,12614.000000,12614.000000,12614.000000,12614.000000,12614,12614.000000,12614.0
mean,15.668384,7.239575,2017.000079,0.345648,0 days 00:38:28.563500872,38.476058,1.0
std,8.472919,3.468836,0.008904,0.475598,0 days 00:28:25.583728325,28.426395,0.0
min,1.000000,1.000000,2017.000000,0.000000,0 days 00:16:00,16.000000,1.0
25%,9.000000,5.000000,2017.000000,0.000000,0 days 00:20:00,20.000000,1.0
50%,15.000000,7.000000,2017.000000,0.000000,0 days 00:27:00,27.000000,1.0
75%,23.000000,10.000000,2017.000000,1.000000,0 days 00:45:00,45.000000,1.0
max,31.000000,12.000000,2018.000000,1.000000,0 days 02:41:00,161.000000,1.0


In [54]:
from pycaret.classification import *
#libreria para generar modelos